<a href="https://colab.research.google.com/github/fjadidi2001/Image_Inpaint/blob/main/Image_inpaint_New_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install fsspec==2024.10.0
!pip install s3fs==2024.10.0
!pip install --upgrade fsspec gcsfs
!pip install --upgrade pip

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.4/77.4 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [2]:
!pip list

Package                            Version
---------------------------------- -------------------
absl-py                            1.4.0
accelerate                         1.1.1
aiobotocore                        2.15.2
aiohappyeyeballs                   2.4.3
aiohttp                            3.11.2
aioitertools                       0.12.0
aiosignal                          1.3.1
alabaster                          1.0.0
albucore                           0.0.19
albumentations                     1.4.20
altair                             4.2.2
annotated-types                    0.7.0
anyio                              3.7.1
argon2-cffi                        23.1.0
argon2-cffi-bindings               21.2.0
array_record                       0.5.1
arviz                              0.20.0
astropy                            6.1.6
astropy-iers-data                  0.2024.11.18.0.35.2
astunparse                         1.6.3
async-timeout                      4.0.3
atpublic           

In [3]:
!pip uninstall gcsfs s3fs fsspec -y
!pip install gcsfs s3fs fsspec

Found existing installation: gcsfs 2024.10.0
Uninstalling gcsfs-2024.10.0:
  Successfully uninstalled gcsfs-2024.10.0
Found existing installation: s3fs 2024.10.0
Uninstalling s3fs-2024.10.0:
  Successfully uninstalled s3fs-2024.10.0
Found existing installation: fsspec 2024.10.0
Uninstalling fsspec-2024.10.0:
  Successfully uninstalled fsspec-2024.10.0
  Using cached s3fs-2024.10.0-py3-none-any.whl.metadata (1.7 kB)
Using cached s3fs-2024.10.0-py3-none-any.whl (29 kB)


In [4]:
!pip install --upgrade tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.3/615.3 MB 42.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 54.4 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.17.1
    Uninstalling tensorboard-2.17.1:
      Successfully uninstalled tensorboard-2.17.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.17.1
    Uninstalling tensorflow-2.17.1:
      Successfully uninstalled tensorflow-2.17.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.18.0 which is incompatible.


In [3]:
!pip install datasets
from datasets import load_dataset

ds = load_dataset("saitsharipov/CelebA-HQ")


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import itertools

In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, Model, mixed_precision
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Step 1: GPU and Memory Configuration
def configure_gpu_memory():
    """
    Configure GPU memory with more robust error handling
    and memory management strategies.
    """
    physical_devices = tf.config.list_physical_devices('GPU')
    if not physical_devices:
        print("No GPU detected. Running on CPU.")
        return False

    try:
        # Reset GPU configurations
        tf.config.experimental.set_memory_growth(physical_devices[0], True)

        # Limit total GPU memory to prevent out-of-memory errors
        tf.config.set_logical_device_configuration(
            physical_devices[0],
            [tf.config.LogicalDeviceConfiguration(memory_limit=4096)]  # Reduced to 4GB
        )

        # Enable mixed precision training for memory efficiency
        mixed_precision.set_global_policy('mixed_float16')

        print("GPU configured successfully.")
        return True
    except Exception as e:
        print(f"GPU configuration error: {e}")
        print("Falling back to CPU or reduced memory mode.")
        return False

In [2]:
# Step 2: Dataset Loading and Preprocessing
def load_dataset(max_images=1000, img_size=128):
    """
    Load and preprocess dataset with memory-efficient approach

    Args:
    - max_images: Maximum number of images to load
    - img_size: Target image size for resizing

    Returns:
    - Processed image numpy array
    """
    from datasets import load_dataset

    try:
        ds = load_dataset("saitsharipov/CelebA-HQ")

        image_paths = []
        for img in tqdm(ds['train']['image'][:max_images]):
            # Resize and normalize images
            resized_img = np.array(
                tf.image.resize(img, [img_size, img_size]) / 255.0,
                dtype=np.float32
            )
            image_paths.append(resized_img)

        return np.array(image_paths)

    except Exception as e:
        print(f"Dataset loading error: {e}")
        return None

In [3]:
# Step 3: Mask Generation
def create_mask(height, width, num_masks=2):
    """
    Generate random masks for image inpainting

    Args:
    - height: Image height
    - width: Image width
    - num_masks: Number of masks to generate

    Returns:
    - Mask numpy array
    """
    mask = np.ones((height, width, 1), dtype=np.float32)

    for _ in range(num_masks):
        y1 = np.random.randint(0, height - height//8)
        x1 = np.random.randint(0, width - width//8)
        h = np.random.randint(height//16, height//8)
        w = np.random.randint(width//16, width//8)
        mask[y1:y1+h, x1:x1+w] = 0.0

    return mask

In [4]:
# Step 4: Data Augmentation
def augment_data(masked_img, mask, img):
    """
    Light data augmentation with minimal overhead

    Args:
    - masked_img: Masked input image
    - mask: Mask array
    - img: Original image

    Returns:
    - Augmented masked image, mask, and original image
    """
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_brightness(img, max_delta=0.1)
    masked_img = tf.image.random_flip_left_right(masked_img)
    mask = tf.image.random_flip_left_right(mask)

    return masked_img, mask, img

In [5]:
# Step 5: Create TensorFlow Dataset
def prepare_dataset(image_paths, batch_size=8, buffer_size=500):
    """
    Create TensorFlow dataset with preprocessing and augmentation

    Args:
    - image_paths: Numpy array of images
    - batch_size: Batch size for training
    - buffer_size: Shuffle buffer size

    Returns:
    - TensorFlow dataset
    """
    def preprocess_image(img):
        mask = create_mask(img.shape[0], img.shape[1])
        masked_img = img * mask
        return masked_img, mask, img

    dataset = tf.data.Dataset.from_tensor_slices(image_paths)
    dataset = dataset.map(preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.map(augment_data, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.shuffle(buffer_size).batch(batch_size).prefetch(tf.data.AUTOTUNE)

    return dataset